In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
import random
import seaborn as sns
import datetime
import pandas as pd
#!pip install sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
import requests
import json
import time
import re

In [3]:
def get_realtime_info(stock_code):
    URL_link = "https://www.citifirst.com.hk/en/data/json/json_realtimedata/code/"+stock_code
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    soup = BeautifulSoup(requests.get(URL_link, headers=headers).content, 'html.parser')

    start_index = str(soup).find("{")
    end_index = str(soup).find("}")
    st = str(soup)[start_index:end_index+1]
    json_str = ""
    for ln in st.split(","):
        if "<" not in ln:
            json_str = json_str + ln + ","
    dic = json.loads(json_str[:-1])

    stock_info = {}
    stock_info['Open'] = dic['open']
    stock_info['High'] = dic['high']
    stock_info['Low'] = dic['low']
    stock_info['Close'] = dic['last']
    stock_info['previous_Close'] = dic['lastc']
    stock_info['turnover'] = dic['turnover']
    stock_info['date_time'] = dic['stimeNoformat']
    return stock_info

# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2023-12-31", real_time=True):
    df_data = None
    df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 

    if real_time:
        stock_info = get_realtime_info(ticker_name)
        open_price, high_price, low_price, current_price = stock_info['Open'], stock_info['High'], stock_info['Low'], stock_info['Close']
        try:
            df_data.at[df_data.index[-1], "Open"] = float(open_price)
            df_data.at[df_data.index[-1], "High"] = float(high_price)
            df_data.at[df_data.index[-1], "Low"] = float(low_price)
            df_data.at[df_data.index[-1], "Close"] = float(current_price)
        except:
            print(stock_info)
            df_data.at[df_data.index[-1], "Open"] = float(current_price)
            df_data.at[df_data.index[-1], "High"] = float(current_price)
            df_data.at[df_data.index[-1], "Low"] = float(current_price)
            df_data.at[df_data.index[-1], "Close"] = float(current_price)
    elif ".HK" in ticker_name:
        print("data may late for 15 minutes")
    # basic calculations such as: daily return, the log of Volume, Moving Average
    df_data['previous_Close'] = df_data['Close'].shift(1)
    df_data['daily_return'] = (df_data['Close']-df_data['previous_Close'])/df_data['previous_Close']
    df_data['Volume_log'] = np.log2(df_data['Volume'])
    MA1, MA2 = 5, 20
    df_data['MA1'] = df_data['Close'].rolling(MA1).mean()
    df_data['MA2'] = df_data['Close'].rolling(MA2).mean()
    return df_data

def get_pair_dates(US_dates, HK_dates):
    pair_indexes = []
    for hk_d in HK_dates:
        idx1 = hk_d
        idx2 = None
        for us_d in US_dates:
            if idx1>=us_d:
                idx2 = us_d
        if idx2 is not None:
            pair_indexes.append( [idx1, idx2] )
    return pair_indexes

def get_merged_df(HK_stock_df, US_stock_df, target_name="Close", offset_row=20):
    #add label for HK stock
    HK_stock_df['next_'+target_name] = HK_stock_df[target_name].shift(-1)
    # 
    US_dates = US_stock_df.index
    HK_dates = HK_stock_df.index
    pair_indexes = get_pair_dates(US_dates, HK_dates)
    # 
    merged_col = [ "HK_date", "HK_Open", "HK_High", "HK_Low", "HK_Close", "HK_Volume", "HK_MA1", "HK_MA2", "HK_next_"+target_name,
                    "US_date", "US_Open", "US_High", "US_Low", "US_Close", "US_Volume", "US_MA1", "US_MA2"
                  ]
    merged_data = []
    for p in pair_indexes[:]:
        hk_d, us_d = p[0], p[1]
        hk_info, us_info = HK_stock_df.loc[hk_d], US_stock_df.loc[us_d]
        hk_use = list(hk_info)[:4] + list(hk_info)[8:-1] # HK_stock_df.columns
        us_use = list(us_info)[:4] + list(us_info)[8:] # US_stock_df.columns
        label_use = list(hk_info)[-1]
        #print( hk_use, us_use, label_use )
        it = [hk_d] + hk_use + [label_use] + [us_d] + us_use
        merged_data.append( it )
    #
    merged_df = pd.DataFrame( merged_data, columns=merged_col )
    return merged_df[offset_row:].copy()

def get_merged_df_update(HK_stock_df, US_stock_df, target_name="Close", offset_row=20):
    # next Open is one of the features
    HK_stock_df['next_Open'] = HK_stock_df['Open'].shift(-1)

    #add label for HK stock
    HK_stock_df['next_'+target_name] = HK_stock_df[target_name].shift(-1)    
    # 
    US_dates = US_stock_df.index
    HK_dates = HK_stock_df.index
    pair_indexes = get_pair_dates(US_dates, HK_dates)
    # 
    merged_col = [ "HK_date", "HK_Open", "HK_High", "HK_Low", "HK_Close", "HK_Volume", "HK_MA1", "HK_MA2", "HK_next_Open", "HK_next_"+target_name,
                    "US_date", "US_Open", "US_High", "US_Low", "US_Close", "US_Volume", "US_MA1", "US_MA2"
                  ]
    merged_data = []
    for p in pair_indexes[:]:
        hk_d, us_d = p[0], p[1]
        hk_info, us_info = HK_stock_df.loc[hk_d], US_stock_df.loc[us_d]
        hk_use = list(hk_info)[:4] + list(hk_info)[8:-1] # HK_stock_df.columns, hk_info
        us_use = list(us_info)[:4] + list(us_info)[8:] # US_stock_df.columns, us_info
        label_use = list(hk_info)[-1]
        #print( hk_use, us_use, label_use )
        it = [hk_d] + hk_use + [label_use] + [us_d] + us_use
        merged_data.append( it )
    #
    merged_df = pd.DataFrame( merged_data, columns=merged_col )
    return merged_df[offset_row:].copy()


# LR model
from sklearn.linear_model import LinearRegression
def train_model(train_X, train_y, printing=True):
    model = LinearRegression().fit(train_X, train_y)

    r_sq = model.score(train_X, train_y)
    if printing:
        print(f"coefficient of determination: {r_sq}")
        print(f"intercept: {model.intercept_}\tslope: {model.coef_}")
    return model

def error_analyze(train_y, y_pred):
    df = pd.DataFrame(columns = ['y_real', 'y_pred'])
    df['y_real'] = train_y
    df['y_pred'] = y_pred
    df['dif'] = (df['y_real'] - df['y_pred'])/df['y_real'] * 100
    df['dif'] = df['dif'].abs()
    return df

def basic_info(df, col='dif'):
    print("max:\t", df[col].max())
    print("min:\t", df[col].min())
    print("median:\t", df[col].median())
    print("mean:\t", df[col].mean())
    print("std:\t", df[col].std())
    print("10%:\t", df[col].quantile(0.10))
    print("25%:\t", df[col].quantile(0.25))
    print("50%:\t", df[col].quantile(0.50))
    print("75%:\t", df[col].quantile(0.75))
    print("90%:\t", df[col].quantile(0.90))
    return

def check_valid(lst):
    for v in lst:
        if math.isinf(v):
            return False
    return True

def predict_current_next_days01(merged_df, train_rows=120, test_row1=-2, test_row2=-1):
    row_data1 = merged_df.iloc[test_row1]
    test_data1, label1 = [list(row_data1)[1:8]+list(row_data1)[10:]], list(row_data1)[8]
    d1 = list(row_data1)[0]
    row_data2 = merged_df.iloc[test_row2]
    test_data2, label2 = [list(row_data2)[1:8]+list(row_data2)[10:]], list(row_data2)[8]
    d2 = list(row_data2)[0]
    #
    train_df = merged_df[-train_rows-2:-2].copy()
    X_data, y_data = [], []
    dates01, dates02 = [], []
    i = 0
    while i<len(train_df):
        row_info = train_df.iloc[i]
        sub_x = list(row_info)[1:8]+list(row_info)[10:]
        sub_y = list(row_info)[8]
        #print(row_info)
        if check_valid(sub_x+[sub_y]):
            X_data.append( sub_x )
            y_data.append( sub_y )
        #
            dates01.append( row_info[0] )
            dates02.append( row_info[9] )
        i += 1
    #
    model = train_model(X_data, y_data)
    y_pred = model.predict(X_data) # error analysis
    error_df = error_analyze(y_data, y_pred)
    error_df['HK_date'] = dates01
    error_df['US_date'] = dates02

    #
    test_y_pred1 = model.predict(test_data1)[0]
    test_y_real1 = label1
    test_y_pred2 = model.predict(test_data2)[0]
    test_y_real2 = label2
    return error_df, (d1, test_y_pred1, test_y_real1), (d2, test_y_pred2, test_y_real2)

def predict_current_next_days02(merged_df, train_rows=120, test_row1=-2, test_row2=-1):
    row_data1 = merged_df.iloc[test_row1]
    test_data1, label1 = [list(row_data1)[1:9]+list(row_data1)[11:]], list(row_data1)[9]
    d1 = list(row_data1)[0] # (d1, test_data1, label1)
    row_data2 = merged_df.iloc[test_row2]
    test_data2, label2 = [list(row_data2)[1:9]+list(row_data2)[11:]], list(row_data2)[9]
    d2 = list(row_data2)[0] # (d2, test_data2, label2)
    # 
    train_df = merged_df[-train_rows-2:-2].copy()
    X_data, y_data = [], []
    dates01, dates02 = [], []
    i = 0
    while i<len(train_df):
        row_info = train_df.iloc[i]
        sub_x = list(row_info)[1:9]+list(row_info)[11:]
        sub_y = list(row_info)[9]
        #print(row_info)
        if check_valid(sub_x+[sub_y]):
            X_data.append( sub_x )
            y_data.append( sub_y )
        #
            dates01.append( row_info[0] )
            dates02.append( row_info[10] )
        i += 1
    #
    model = train_model(X_data, y_data)
    y_pred = model.predict(X_data) # error analysis
    error_df = error_analyze(y_data, y_pred)
    error_df['HK_date'] = dates01
    error_df['US_date'] = dates02

    #
    test_y_pred1 = model.predict(test_data1)[0]
    test_y_real1 = label1
    #test_y_pred2 = model.predict(test_data2)[0]
    #test_y_real2 = label2
    return error_df, (d1, test_y_pred1, test_y_real1), (d2, test_data2, label2) #(d2, test_y_pred2, test_y_real2)

In [4]:
stocks_info = [
    ('BABA', '9988.HK', 1),
    ('BIDU', '9888.HK', 1),
    ('JD', '9618.HK', 0.5 * 7.8),
    ('MPNGY', '3690.HK', 0.5 * 7.8),
    ('NTES', '9999.HK', 0.2 * 7.8),
	('LI', '2015.HK', 0.5 * 7.8),
	('XPEV', '9868.HK', 0.5*7.8),	
	('BILI', '9626.HK', 1 * 7.8),
	('TCOM', '9961.HK', 1 * 7.8),
	('YUMC', '9987.HK', 1*7.8),
	('EDU', '9901.HK', 0.1*7.8), 
    ('NIO', '9866.HK', 1 * 7.8),
	('ZTO', '2057.HK', 1*7.8),
    ('BEKE', '2423.HK', 0.5*7.8),
    ('ZH', '2390.HK', 3 * 7.8), 
    ('WB', '9898.HK', 1*7.8),
    ('MNSO', '9896.HK', 0.5*7.8),
    ('ZLAB', '9688.HK', 0.5*7.8),
    ('TME', '1698.HK', 1*7.8),
]

st, et = "2021-01-01", "2023-08-31"

In [23]:
predicted_data = []
predicted_features = ['Close', 'High', 'Low']

mode = 2

for si in stocks_info[:]:
    US_stock_code, HK_stock_code, _ = si
    new_name = US_stock_code + "_" + HK_stock_code
    US_stock_df = get_df_data(US_stock_code, st, et, False)
    HK_stock_df = get_df_data(HK_stock_code, st, et, True)

    each_row = [new_name]

    
    if mode == 1:
        for tn in predicted_features:
            target_name = tn
            merged_df = get_merged_df(HK_stock_df.copy(), US_stock_df.copy(), target_name, 20)
            error_df, pred_it1, it2 = predict_current_next_days01(merged_df, 200, -2, -1)
            err1, err2 = round(error_df['dif'].median(), 3), round(error_df['dif'].mean(), 3)
            print( US_stock_code, HK_stock_code, tn, pred_it1, err1, err2, it2  )
            each_row = each_row + [ pred_it1[1], pred_it1[2], err2, it2[1] ]
    elif mode == 2:
        for tn in predicted_features:
            target_name = tn
            merged_df = get_merged_df_update(HK_stock_df.copy(), US_stock_df.copy(), target_name, 20)
            error_df, pred_it1, it2 = predict_current_next_days02(merged_df, 200, -2, -1) # 
            err1, err2 = round(error_df['dif'].median(), 3), round(error_df['dif'].mean(), 3)
            print( US_stock_code, HK_stock_code, tn, pred_it1, err1, err2  )
            each_row = each_row + [ pred_it1[1], pred_it1[2], err2 ]
    
    predicted_data.append( each_row ) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
coefficient of determination: 0.9760617994926977
intercept: 6.414589456541336	slope: [ 0.02904152  0.18758273 -0.13695947 -0.20237467  0.12927154 -0.03335859
  0.23770799  0.6931417  -0.26941008  0.41076385  0.41502982 -0.21305726
 -0.35202529  0.10566733 -0.24501929]
BABA 9988.HK Close (Timestamp('2023-08-17 00:00:00'), 89.34990048981487, 89.25) 1.131 1.53
coefficient of determination: 0.9815476394211395
intercept: 0.2042564905758013	slope: [ 0.0200395   0.23969382 -0.16555371 -0.16351152  0.25335821  0.03145543
  0.07776734  0.91477883 -0.22969134  0.36243489  0.1720711  -0.22946916
 -0.16715821  0.03483308 -0.08684416]
BABA 9988.HK High (Timestamp('2023-08-17 00:00:00'), 90.84366276288742, 89.25) 1.094 1.281
coefficient of determination: 0.992869713324505
intercept: 6.79239272566349	slope: [ 0.03122639 -0.01715866 -0.07191526 -0.022886   -0.203500

In [26]:
predicted_data

###
col_names = [
            "stock-name", 
            "close_pred", "close_real", "close_err_range%", "nd_close",
            "high_pred", "high_real", "high_err_range%", "nd_high",
            "low_pred", "low_real", "low_err_range%", "nd_low",            
            ]
if mode == 2:
    col_names = [
            "stock-name", 
            "close_pred", "close_real", "close_err_range%", 
            "high_pred", "high_real", "high_err_range%", 
            "low_pred", "low_real", "low_err_range%",    
            ]


round_dic = {'close_pred': 2, 'high_pred': 2, 'low_pred': 2, 
             'close_err_range%': 2, 'high_err_range%':2, 'low_err_range%':2, 
             'nd_close':2, 'nd_high':2, 'nd_low':2,
              'daily_return%':2 }
stock_df = pd.DataFrame( predicted_data, columns=col_names )

close_real, high_real, low_real = [], [], []
previous_Close, turnover = [], []
for si in stocks_info[:]:
    US_stock_code, HK_stock_code, _ = si
    newest_info = get_realtime_info(HK_stock_code)
    close_real.append( float(newest_info['Close']) )
    try:
        high_real.append( float(newest_info['High']) )
        low_real.append( float(newest_info['Low']) )
    except:
        high_real.append( newest_info['High'] )
        low_real.append( newest_info['Low'] )
    previous_Close.append( float(newest_info['previous_Close']) )
    turnover.append( newest_info['turnover'] )

stock_df['close_real'] = close_real
stock_df['high_real'] = high_real
stock_df['low_real'] = low_real
stock_df['previous_Close'] = previous_Close
stock_df['turnover'] = turnover

show_cols = [
            "stock-name", 'previous_Close', 'price_change', 'daily_return%',
            "close_pred", "close_real", "close_err_range%", 
            "high_pred", "high_real", "high_err_range%", 
            "low_pred", "low_real", "low_err_range%", 'turnover',
            "nd_close", "nd_high", "nd_low", 
        ]
if mode == 2:
    show_cols = show_cols[:-3]

stock_df['price_change'] = stock_df['close_real'] - stock_df['previous_Close']
stock_df['daily_return%'] = (stock_df['close_real'] - stock_df['previous_Close']) / stock_df['previous_Close'] * 100

stock_df.round(round_dic)[ show_cols ] 

,stock-name,previous_Close,price_change,daily_return%,close_pred,close_real,close_err_range%,high_pred,high_real,high_err_range%,low_pred,low_real,low_err_range%,turnover
0,BABA_9988.HK,90.10,-3.10,-3.44,89.35,87.00,1.53,90.84,89.85,1.28,88.17,86.80,0.86,3.71B
1,BIDU_9888.HK,129.60,-3.50,-2.70,128.43,126.10,1.72,130.54,129.70,1.45,126.64,125.80,1.05,573.01M
2,JD_9618.HK,138.60,-7.40,-5.34,135.27,131.20,1.82,137.54,136.40,1.26,133.06,131.20,1.07,2.29B
3,MPNGY_3690.HK,133.60,-3.70,-2.77,132.02,129.90,2.02,134.78,133.80,1.49,129.73,129.40,1.30,2.31B
4,NTES_9999.HK,161.90,-5.40,-3.34,158.96,156.50,1.51,161.07,161.00,1.03,156.79,156.00,1.00,599.98M
5,LI_2015.HK,164.00,-8.60,-5.24,163.03,155.40,2.12,166.93,165.20,1.52,160.93,153.80,1.29,1.45B
6,XPEV_9868.HK,65.30,-4.30,-6.58,64.12,61.00,2.91,64.72,63.45,2.12,61.86,60.65,1.67,1.08B
7,BILI_9626.HK,124.00,-7.10,-5.73,123.00,116.90,2.47,126.62,128.00,1.82,118.78,116.80,1.43,624.44M
8,TCOM_9961.HK,311.20,-11.60,-3.73,301.99,299.60,1.54,307.01,305.00,1.08,298.56,298.20,1.00,428.76M
9,YUMC_9987.HK,428.80,-12.20,-2.85,431.90,416.60,1.25,436.63,427.80,1.00,422.49,416.00,0.80,135.11M
